In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from keras import regularizers
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adagrad
from keras.utils import np_utils
from numpy.random import seed
from tensorflow import set_random_seed

from keras import backend as K

In [3]:
def get_feables(df):
    if 'date' in df.columns:
        features = df.drop(columns=['result', 'date'])
    else:
        features = df.drop(columns=['result'])
    encoder = LabelEncoder()
    labels = np_utils.to_categorical(-encoder.fit_transform(df.result.copy()) + 2)
    return features, labels

In [4]:
def profit_(y_true, y_pred, threshold):
    selection = K.cast((y_pred > threshold),'float32')
    odds_true = K.clip(y_true, 0., np.inf) - 1
    return K.sum( selection * odds_true, axis=0), K.sum(selection, axis=0)

def bet_metric(th=0.05):
    def profit(y_true, y_pred):
        true_odds = K.abs(y_true)
        condition = (1./true_odds + th)
        return K.sum(profit_(y_true, y_pred, condition)[0])
    def margin(y_true, y_pred):
        true_odds = K.abs(y_true)
        condition = (1./true_odds + th)
        pr = profit_(y_true, y_pred, condition)
        return K.sum(pr[0]) / K.sum(pr[1])
    return profit, margin

def abs_bet_metric(th=0.9):
    def abs_profit(y_true, y_pred):
        return K.sum(profit_(y_true, y_pred, th)[0])
    def abs_margin(y_true, y_pred):
        pr = profit_(y_true, y_pred, th)
        return K.sum(pr[0]) / K.sum(pr[1])
    return abs_profit, abs_margin

def bet_metrics(th=0.05, abs_th=0.9):
    return list(bet_metric(th)) + list(abs_bet_metric(abs_th))

def d_performance(th=0.05, name=None):
    def perf_metric(y_pred, y_true):
        ypred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        ytrue = tf.convert_to_tensor(y_true, dtype=tf.float32)
        true_odds = K.abs(ytrue)
        condition = (1./true_odds + th)
        pr, co = profit_(ytrue, ypred, condition)
        if y_pred.shape[1] == 4:
            idx = ['home','draw','away','no-bet']
        else:
            idx = ['home','draw','away']
        profit = pd.Series(K.eval(pr), index=idx)
        count = pd.Series(K.eval(co), index=idx)
        profit.name = 'Profit '
        count.name = 'Count '
        if not name is None:
            profit.name = profit.name + name
            count.name = count.name + name

        return profit, count
    return perf_metric

def d_absolute_performance(th=0.9, name=None):
    def abs_perf_metric(y_pred, y_true):
        ypred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        ytrue = tf.convert_to_tensor(y_true, dtype=tf.float32)
        pr, co = profit_(ytrue, ypred, th)
        if y_pred.shape[1] == 4:
            idx = ['home','draw','away','no-bet']
        else:
            idx = ['home','draw','away']
        profit = pd.Series(K.eval(pr), index=idx)
        count = pd.Series(K.eval(co), index=idx)
        profit.name = '|Profit| '
        count.name = '|Count| '
        if not name is None:
            profit.name = profit.name + name
            count.name = count.name + name

        return profit, count
    return abs_perf_metric

def performance(model, X, c, th1 = 0.05, th2=0.9, name=None):
    return pd.concat([pd.concat(d_performance(th1,name=name)(model.predict(X), c.values),axis=1),
                      pd.concat(d_absolute_performance(th2,name=name)(model.predict(X), c.values),axis=1)], axis=1)

In [8]:
def construct_model(input_classes, output_classes=3, loss='categorical_crossentropy', metrics=['accuracy'], 
                    normalize=False, hidden_layer=[10], activation='softmax'):
    K.clear_session()

    seed(42)
    set_random_seed(42)

    model = Sequential()
    model.add(Dense(hidden_layer[0], input_dim=input_classes,
                    kernel_regularizer=regularizers.l1(0.001)
                   ))
    if normalize:
        model.add(BatchNormalization())
    for i in range(1, len(hidden_layer)):
        model.add(Dense(hidden_layer[i], 
                    kernel_regularizer=regularizers.l1(0.001)
                 ))
        if normalize:
            model.add(BatchNormalization())
        
    model.add(Dense(output_classes, 
                    kernel_regularizer=regularizers.l1(0.001),
                    activation = activation
                   ))
       
    model.compile(loss=loss, 
                  optimizer=Adagrad(0.1),
                  metrics=metrics
                 )

    return model

In [ ]:
def get_model(loss='categorical_crossentropy', output_classes=3, metrics=['accuracy'], hidden_layer = [10], normalize=False):
    K.clear_session()

    seed(42)
    set_random_seed(42)

    model = Sequential()
    if len(hidden_layer):
        model.add(Dense(hidden_layer[0], input_dim=X_train.shape[1],
                        #kernel_regularizer=regularizers.l1(0.001)
                       ))
        if normalize:
            model.add(BatchNormalization())
        for i in range(1, len(hidden_layer)):
            model.add(Dense(hidden_layer[i], 
                            #kernel_regularizer=regularizers.l1(0.001)
                           ))
        if normalize:
            model.add(BatchNormalization())
        model.add(Dense(output_classes, activation='softmax'))
    else:
        model.add(Dense(output_classes, input_dim=X_train.shape[1],
                        #kernel_regularizer=regularizers.l1(0.001),
                        activation = 'softmax'
                       ))
       
    model.compile(loss=loss, 
                  optimizer=Adagrad(), # 0.1
                  metrics=metrics
                 )

    return model